# Limpeza e Tratamento dos Dados

## Imports

In [1]:
import pandas as pd
import inflection

## Helper Variables

In [2]:
RAW_DATA_PATH = f"../dataset/zomato.csv"

COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zealand",
    162: "Philippines",
    166: "Qatar",
    184: "Singapore",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}

COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

## Helper Functions

In [3]:
def country_name(country_id):
  return COUNTRIES[country_id]

def color_name(color_code):
  return COLORS[color_code]

def create_price_type(price_range):
  if price_range == 1:
    return "cheap"
  elif price_range == 2:
    return "normal"
  elif price_range == 3:
    return "expensive"
  else:
    return "gourmet"

def adjust_columns_order(dataframe):
    df = dataframe.copy()

    new_cols_order = [
        "restaurant_id",
        "restaurant_name",
        "country",
        "city",
        "address",
        "locality",
        "locality_verbose",
        "longitude",
        "latitude",
        "cuisines",
        "price_type",
        "average_cost_for_two",
        "currency",
        "has_table_booking",
        "has_online_delivery",
        "is_delivering_now",
        "aggregate_rating",
        "rating_color",
        "color_name",
        "rating_text",
        "votes",
    ]

    return df.loc[:, new_cols_order]

def process_data(file_path):
    df = pd.read_csv(file_path)
    df = df.dropna()
    df = rename_columns(df)
    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: create_price_type(x))
    df["country"] = df.loc[:, "country_code"].apply(lambda x: country_name(x))
    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x))
    df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])
    df = df.drop_duplicates()
    df = adjust_columns_order(df)
    df.to_csv("../dataset/processed.csv", index=False)
    return df

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

## Load Data

In [4]:
df_raw = pd.read_csv(RAW_DATA_PATH)
df1 = df_raw.copy()
df1 = rename_columns(df1)
df1 = df1.dropna()
df2 = process_data(RAW_DATA_PATH)
df2.head()

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,1100,Botswana Pula(P),1,0,0,4.6,3F7E00,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,3100,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,469
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,800,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,867
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,700,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,858
5,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,700,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,930


# Geral

1. Quantos restaurantes únicos estão registrados?

In [24]:
df2['restaurant_id'].nunique()

6929

2. Quantos países únicos estão registrados?

In [26]:
df2['country'].nunique()

15

3. Quantas cidades únicas estão registradas?

In [27]:
df2['city'].nunique()

125

4. Qual o total de avaliações feitas?

In [29]:
df2['votes'].sum()

4194533

5. Qual o total de tipos de culinária registrados?

In [31]:
df2['cuisines'].nunique()

165

# País

1. Qual o nome do país que possui mais cidades registradas?

In [36]:
(df2.loc[:, ['country','city']].groupby('country')
 .nunique()
 .sort_values('city', ascending=False)
 .reset_index()).iloc[0, 0]

'India'

2. Qual o nome do país que possui mais restaurantes registrados?

In [9]:
(df2.loc[:, ['restaurant_id','country']].groupby('country')
 .count()
 .sort_values('restaurant_id', ascending=False)
 .reset_index()).iloc[0, 0]

'India'

3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 
registrados?


In [13]:
restaurantes_nota_4 = df2['price_type'] == 'gourmet'
df_auxp03 = df2.loc[restaurantes_nota_4, ['restaurant_id', 'country']].groupby('country').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxp03.iloc[0, 0]

'United States of America'

4. Qual o nome do país que possui a maior quantidade de tipos de culinária 
distintos?

In [14]:
df_auxp04 = df2.loc[:, ['country','cuisines']].groupby('country').nunique().sort_values('cuisines', ascending=False).reset_index()
df_auxp04.iloc[0, 0]

'India'

5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [15]:
df_auxp05 = df2.loc[:, ['country','votes']].groupby('country').sum().sort_values('votes', ascending=False).reset_index()
df_auxp05.iloc[0, 0]

'India'

6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem 
entrega?

In [18]:
restaurantes_entregando = df2['is_delivering_now'] == 1
df_auxp06 = df2.loc[restaurantes_entregando, ['restaurant_id', 'country']].groupby('country').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxp06.iloc[0, 0]

'India'

7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam 
reservas?

In [12]:
restaurantes_reservas = df2['has_table_booking'] == 1
df_auxp07 = df2.loc[restaurantes_reservas, ['restaurant_id', 'country']].groupby('country').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxp07.iloc[0, 0]

'India'

8. Qual o nome do país que possui, na média, a maior quantidade de avaliações 
registrada?

In [20]:
df_auxp08 = df2.loc[:, ['votes', 'country']].groupby('country').mean().sort_values('votes', ascending=False).reset_index()
df_auxp08.iloc[0, 0]

'Indonesia'

9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [22]:
df_auxp09 = df2.loc[:, ['aggregate_rating', 'country']].groupby('country').mean().sort_values('aggregate_rating', ascending=False).reset_index()
df_auxp09.iloc[0, 0]

'Indonesia'

10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [23]:
df_auxp10 = df2.loc[:, ['aggregate_rating', 'country']].groupby('country').mean().sort_values('aggregate_rating', ascending=True).reset_index()
df_auxp10.iloc[0, 0]

'Brazil'

11. Qual a média de preço de um prato para dois por país?

In [24]:
df_auxp11 = df2.loc[:, ['average_cost_for_two', 'country']].groupby('country').mean().sort_values('average_cost_for_two', ascending=False).reset_index()
df_auxp11

,country,average_cost_for_two
0,Indonesia,303000.000000
1,Australia,138959.783333
2,Sri Lanka,2579.375000
3,Philippines,1227.825000
4,India,704.400514
5,South Africa,339.228324
6,Qatar,174.000000
7,United Arab Emirates,153.716667
8,Singapore,141.437500
9,Brazil,138.812500


# Cidade

1. Qual o nome da cidade que possui mais restaurantes registrados?

In [26]:
df_auxc01 = df2.loc[:, ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxc01.iloc[0, 0]

'Abu Dhabi'

2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [28]:
restaurantes_acima_de_4 = df2['aggregate_rating'] > 4
df_auxc02 = df2.loc[restaurantes_acima_de_4, ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxc02.iloc[0, 0]

'Bangalore'

3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [29]:
restaurantes_abaixo_250 = df2['aggregate_rating'] < 2.5
df_auxc03 = df2.loc[restaurantes_abaixo_250, ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxc03.iloc[0, 0]

'Gangtok'

4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [30]:
df_auxc04 = df2.loc[:, ['average_cost_for_two', 'city']].groupby('city').mean().sort_values('average_cost_for_two', ascending=False).reset_index()
df_auxc04.iloc[0, 0]

'Adelaide'

5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [31]:
df_auxc05 = df2.loc[:, ['cuisines', 'city']].groupby('city').nunique().sort_values('cuisines', ascending=False).reset_index()
df_auxc05.iloc[0, 0]

'Birmingham'

6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [33]:
restaurantes_reservas = df2['has_table_booking'] == 1
df_auxc06 = df2.loc[restaurantes_reservas, ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxc06.iloc[0, 0]

'Bangalore'

7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [35]:
restaurantes_entregando = df2['is_delivering_now'] == 1
df_auxc07 = df2.loc[restaurantes_entregando, ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxc07.iloc[0, 0]

'Vadodara'

8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [37]:
restaurantes_online = df2['has_online_delivery'] == 1
df_auxc08 = df2.loc[restaurantes_online, ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxc08.iloc[0, 0]

'Bhopal'

# Restaurantes

1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [26]:
df_auxr01 = df2.loc[:, ['restaurant_name', 'votes']].groupby('restaurant_name').max().sort_values('votes', ascending=False).reset_index()
df_auxr01.iloc[0, 0]

'Bawarchi'

2. Qual o nome do restaurante com a maior nota média?

In [14]:
maior_nota = df2['aggregate_rating'] == df2['aggregate_rating'].max()
df_auxr02 = df2.loc[maior_nota, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(by='restaurant_id')
df_auxr02.iloc[0,0]


'Indian Grill Room'

3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [41]:
maior_valor = df2['average_cost_for_two'] == df2['average_cost_for_two'].max()
df_auxr03 = df2.loc[maior_valor, ['restaurant_name', 'restaurant_id', 'average_cost_for_two']].sort_values(by='restaurant_id')
df_auxr03.iloc[0,0]

"d'Arry's Verandah Restaurant"

4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [44]:
menor_avaliacao = (df2['cuisines'] == 'Brazilian') & (df2['aggregate_rating'] == df2['aggregate_rating'].min())
df_auxr04 = df2.loc[menor_avaliacao, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(by='restaurant_id')
df_auxr04.iloc[0, 0]


'Loca Como tu Madre'

5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [10]:
restaurantes_selecionados = (df2['cuisines'] == 'Brazilian') & (df2['country'] == 'Brazil') & (df2['aggregate_rating'] == df2['aggregate_rating'].max())
df_auxr05 = df2.loc[restaurantes_selecionados, ['restaurant_name', 'aggregate_rating', 'restaurant_id']].sort_values(by='restaurant_id')
df_auxr05.iloc[0, 0]

'Braseiro da Gávea'

6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [11]:
df2.loc[:, ['has_online_delivery', 'votes']].groupby('has_online_delivery').mean().sort_values('votes', ascending=False).reset_index()

,has_online_delivery,votes
0,1,838.821664
1,0,479.421018


7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [12]:
df2.loc[:, ['has_table_booking', 'average_cost_for_two']].groupby('has_table_booking').mean().sort_values('average_cost_for_two', ascending=False).reset_index()

,has_table_booking,average_cost_for_two
0,1,69998.423810
1,0,3488.596866


8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as 
churrascarias americanas (BBQ)?

In [16]:
lines = (df2['country'] == 'United States of America') & ((df2['cuisines'] == 'Japanese') | (df2['cuisines'] == 'BBQ'))
# Condição ou: se os restaurantes de cozinha japonesa e BBQ forem dos EUA, eles precisam estar entre parênteses para obedecer à condição OU.

df2.loc[lines, ['cuisines', 'average_cost_for_two']].groupby('cuisines').mean().sort_values('average_cost_for_two', ascending=False).reset_index()

,cuisines,average_cost_for_two
0,Japanese,56.406250
1,BBQ,39.642857


# Tipos de Culinária

1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [17]:
maior_avaliacao = (df2['cuisines'] == 'Italian') & (df2['aggregate_rating'] == df2['aggregate_rating'].max())
df_auxn01 = df2.loc[maior_avaliacao, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(by='restaurant_id')
df_auxn01.iloc[0, 0]

'Darshan'

2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [18]:
menor_avaliacao = (df2['cuisines'] == 'Italian') & (df2['aggregate_rating'] == df2['aggregate_rating'].min())
df_auxn02 = df2.loc[menor_avaliacao, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(by='restaurant_id')
df_auxn02.iloc[0, 0]



'Avenida Paulista'

3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [19]:
maior_avaliacao = (df2['cuisines'] == 'American') & (df2['aggregate_rating'] == df2['aggregate_rating'].max())
df_auxn03 = df2.loc[maior_avaliacao, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(by='restaurant_id')
df_auxn03.iloc[0, 0]

'Burger & Lobster'

4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [20]:
menor_avaliacao = (df2['cuisines'] == 'American') & (df2['aggregate_rating'] == df2['aggregate_rating'].min())
df_auxn04 = df2.loc[menor_avaliacao, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(by='restaurant_id')
df_auxn04.iloc[0, 0]


'Alston Bar & Beef'

5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do 
restaurante com a maior média de avaliação?? 

In [10]:
restaurantes_arab = df2['cuisines'] == 'Arabian' 
df_auxn05 = df2.loc[restaurantes_arab, ['restaurant_name', 'aggregate_rating', 'restaurant_id']].sort_values('aggregate_rating', ascending=False).reset_index()
df_auxn05.iloc[0, 1]

'Mandi@36'

6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do 
restaurante com a menor média de avaliação??

In [10]:
restaurantes_arab = df2['cuisines'] == 'Arabian' 
df_auxn05 = df2.loc[restaurantes_arab, ['restaurant_name', 'aggregate_rating', 'restaurant_id']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).reset_index()
df_auxn05.iloc[0, 1],

'Raful'

7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do 
restaurante com a maior média de avaliação???

In [15]:
maior_avaliacao = (df2['cuisines'] == 'Japanese') & (df2['aggregate_rating'] == df2['aggregate_rating'].max())
df_auxn07 = df2.loc[maior_avaliacao, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(by='restaurant_id')
df_auxn07.iloc[0, 0]

'Sushi Samba'

8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do 
restaurante com a menor média de avaliação?)?

In [16]:
menor_avaliacao = (df2['cuisines'] == 'Japanese') & (df2['aggregate_rating'] == df2['aggregate_rating'].min())
df_auxn08 = df2.loc[menor_avaliacao, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(by='restaurant_id')
df_auxn08.iloc[0, 0]

'Banzai Sushi'

9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do 
restaurante com a maior média de avaliação?


In [8]:
home_made = df2['cuisines'] == 'Home-made'
df_auxn09 = df2.loc[home_made, ['restaurant_name', 'aggregate_rating', 'restaurant_id']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).reset_index()
df_auxn09.iloc[0, 1]



'Kanaat Lokantası'

10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do 
restaurante com a menor média de avaliação?

In [9]:
home_made = df2['cuisines'] == 'Home-made'
df_auxn10 = df2.loc[home_made, ['restaurant_name', 'aggregate_rating', 'restaurant_id']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).reset_index()
df_auxn10.iloc[0, 1]

'GurMekan Restaurant'

11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas 
pessoas?

In [12]:
df_auxn11 = df2.loc[:, ['cuisines', 'average_cost_for_two']].groupby('cuisines').mean().sort_values('average_cost_for_two', ascending=False).reset_index()
df_auxn11.head(1)

,cuisines,average_cost_for_two
0,Modern Australian,1.470693e+06


12. Qual o tipo de culinária que possui a maior nota média?

In [16]:
df_auxn12 = df2.loc[:, ['cuisines', 'aggregate_rating']].groupby('cuisines').mean().sort_values('aggregate_rating', ascending=False).reset_index()
df_auxn12.head(1)

,cuisines,aggregate_rating
0,Others,4.9


13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos 
online e fazem entregas?

In [18]:
restaurantes = (df2['has_online_delivery'] == 1) & (df2['is_delivering_now'] == 1)
df_auxn13 = df2.loc[restaurantes, ['cuisines', 'restaurant_id']].groupby('cuisines').count().sort_values('restaurant_id', ascending=False).reset_index()
df_auxn13.head(1)

,cuisines,restaurant_id
0,North Indian,317
